In [1]:
!pip install tableauserverclient

     -------------------------------------- 142.0/142.0 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 124.2/124.2 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.0.7 which is incompatible.


In [15]:
import tableauserverclient as TSC
import csv
import pandas as pd
import io

In [16]:
portal_list=['890Portal']

In [17]:
server = TSC.Server('http://10.100.252.218')

In [18]:
USERNAME = "TableauAdmin"
PASSWORD = "CapVpnCluster$20"

In [14]:
for portal in portal_list:
    tableau_auth = TSC.TableauAuth(USERNAME,PASSWORD,portal)
    server.auth.sign_in(tableau_auth)

    all_projects, pagination_item = server.projects.get()
    prj_name = [prj.name for prj in all_projects]
    
    all_workbooks, pagination_item = server.workbooks.get()
    wrk_name = [[w_book.name,w_book.id] for w_book in all_workbooks]
    
    work_list = [[w_book.name,w_book.id,w_book.project_id,w_book._project_name,w_book.owner_id] for w_book in all_workbooks]
    all_views, pagination_item = server.views.get()
    view_list = [[view.name,view.workbook_id] for view in all_views]

    final_data = [[item1[3],item1[0],item2[0],item1[1],item1[2],item1[4]] for item1 in work_list for item2 in view_list if item1[1]==item2[1]]

    df_final_data  =pd.DataFrame(final_data,columns=['Project Name','Workbook Name','Sheet Name','WORKBOOKID','Project_ID','Owner Id/Owner Name'])  

    all_workbooks_items, pagination_item = server.workbooks.get()
    wbs = []
    wbs_con = []
    for wb in all_workbooks_items:
        server.workbooks.populate_connections(wb)
    
        for connection in wb.connections:

            wbs_con.append({
                "WORKBOOKID": wb.id,
                "WORKBOOKNAME": wb.name,
                "CONNECTID": connection.id,
                "CONNECTTYPE": connection.connection_type,
                "SERVERADDRESS": connection.server_address,
                "SERVERPORT": connection.server_port,
                "USERNAME": connection.username,
                "DATASOURCEID": connection.datasource_id,
                "DATASOURCENAME": connection.datasource_name,
    #             "View": connection.view_name
                })
    wb_con_df = pd.DataFrame(wbs_con)

    df_1 = wb_con_df[['WORKBOOKID','CONNECTTYPE','DATASOURCENAME']]

    pivot = df_1.pivot_table(index=['WORKBOOKID'],
                           values=['DATASOURCENAME','CONNECTTYPE'],
                           aggfunc=lambda x: list(x))# apply function to convert to list by using to_list
    # pivot.to_excel("Data_Source_Pivote.xlsx")
    
    pivot_1 = df_final_data.pivot_table(index=['WORKBOOKID'],
                           values=['Project Name','Workbook Name','Sheet Name','Project_ID','Owner Id/Owner Name'],
                           aggfunc=lambda x: list(x))
    
    merge_results = pd.merge(pivot, pivot_1, on="WORKBOOKID")
    explode_1 = merge_results.explode(['Project Name','Workbook Name','Sheet Name','Project_ID','Owner Id/Owner Name'])
    explode_1 = explode_1[['Project Name','Workbook Name','Sheet Name','DATASOURCENAME','CONNECTTYPE','Owner Id/Owner Name']]
    explode_1.reset_index(drop=True, inplace=True)
    explode_1.rename(columns ={'DATASOURCENAME':'Data source Name','CONNECTTYPE':'Connection type'}, inplace = True)

    unique_values_set = set(explode_1['Project Name'])
    unique_list = [*unique_values_set, ]
    
    temp3 = [x for x in prj_name if x not in unique_list]
    
    df_temp =pd.DataFrame(temp3,columns=['Project Name'])
    
    frames=[explode_1,df_temp]
    result = pd.concat(frames)
    result.to_excel('Tableau_Meta_Data_'+portal+'.xlsx',index=False)

In [63]:
def extract_tableau_data(username, password, portals):
    for portal in portals:
        tableau_auth = TSC.TableauAuth(username, password, portal)
        server.auth.sign_in(tableau_auth)

        with server.auth.sign_in(tableau_auth):
            print(f"Logged in to {portal} successfully!")

            # Determine and use the highest available API version for the server
            highest_version = server.version
            print(f"Using API version: {highest_version}")

            # Get all projects
            all_projects, pagination_item = server.projects.get()
            prj_name = [prj.name for prj in all_projects]
            
            # Get all workbooks
            all_workbooks, pagination_item = server.workbooks.get()
            work_list = [[w_book.name, w_book.id, w_book.project_id, w_book.project_name, w_book.owner_id] for w_book in all_workbooks]
            
            # Get all views
            all_views, pagination_item = server.views.get()
            view_list = [[view.name, view.workbook_id] for view in all_views]
            
            final_data = [[item1[3], item1[0], item2[0], item1[1], item1[2], item1[4]] for item1 in work_list for item2 in view_list if item1[1] == item2[1]]
            df_final_data = pd.DataFrame(final_data, columns=['Project Name', 'Workbook Name', 'Sheet Name', 'WORKBOOKID', 'Project_ID', 'Owner Id/Owner Name'])
            
            # Get workbook connections
            all_workbooks_items, pagination_item = server.workbooks.get()
            wbs_con = []
            for wb in all_workbooks_items:
                server.workbooks.populate_connections(wb)
                for connection in wb.connections:
                    wbs_con.append({
                        "WORKBOOKID": wb.id,
                        "WORKBOOKNAME": wb.name,
                        "CONNECTID": connection.id,
                        "CONNECTTYPE": connection.connection_type,
                        "SERVERADDRESS": connection.server_address,
                        "SERVERPORT": connection.server_port,
                        "USERNAME": connection.username,
                        "DATASOURCEID": connection.datasource_id,
                        "DATASOURCENAME": connection.datasource_name,
                    })
            
            wb_con_df = pd.DataFrame(wbs_con)
            df_1 = wb_con_df[['WORKBOOKID', 'CONNECTTYPE', 'DATASOURCENAME']]
            
            pivot = df_1.pivot_table(index=['WORKBOOKID'],
                                     values=['DATASOURCENAME', 'CONNECTTYPE'],
                                     aggfunc=lambda x: list(x))
            
            pivot_1 = df_final_data.pivot_table(index=['WORKBOOKID'],
                                                values=['Project Name', 'Workbook Name', 'Sheet Name', 'Project_ID', 'Owner Id/Owner Name'],
                                                aggfunc=lambda x: list(x))
            
            merge_results = pd.merge(pivot, pivot_1, on="WORKBOOKID")
            explode_1 = merge_results.explode(['Project Name', 'Workbook Name', 'Sheet Name', 'Project_ID', 'Owner Id/Owner Name'])
            explode_1 = explode_1[['Project Name', 'Workbook Name', 'Sheet Name', 'DATASOURCENAME', 'CONNECTTYPE', 'Owner Id/Owner Name']]
            explode_1.reset_index(drop=True, inplace=True)
            explode_1.rename(columns={'DATASOURCENAME': 'Data source Name', 'CONNECTTYPE': 'Connection type'}, inplace=True)
            
            unique_values_set = set(explode_1['Project Name'])
            unique_list = [*unique_values_set, ]
            temp3 = [x for x in prj_name if x not in unique_list]
            
            df_temp = pd.DataFrame(temp3, columns=['Project Name'])
            
            frames = [explode_1, df_temp]
            result = pd.concat(frames)
            result.to_excel(f'Tableau_Meta_Data_{portal}.xlsx', index=False)
            
            # Export all Datasource information to CSV
            wb_con_df.to_csv(f'Datasource_Info_{portal}.csv', index=False)
            
            # Extract CSV data for each view
            field_data = []
            for view in all_views:
                try:
                    server.views.populate_csv(view)  # Populate the CSV data for the view
                    csv_generator = view.csv
                    csv_data = ''.join(list(csv_generator))  # Convert the generator to a string
                    print(f'The csv data : {csv_data}')
                    
                    # Parse csv_data to understand fields used in the view
                    csv_reader = pd.read_csv(io.StringIO(csv_data))
                    for column in csv_reader.columns:
                        field_data.append({
                            'View Name': view.name,
                            'Workbook ID': view.workbook_id,
                            'Field Name': column,
                            'Datasource Name': "Unknown"  # Adjust this as needed
                        })
                except TSC.ServerResponseError as e:
                    print(f"Error extracting CSV data for view {view.name}: {e}")
                except Exception as e:
                    print(f"An unexpected error occurred while processing view {view.name}: {e}")
            
            df_field_data = pd.DataFrame(field_data)
            df_field_data.to_csv(f'Fields_Used_In_Views_{portal}.csv', index=False)


In [64]:
extract_tableau_data(USERNAME, PASSWORD, portal_list)

Logged in to 890Portal successfully!
Using API version: 2.7
Error extracting CSV data for view Reach: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Promotion Effect: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Conversion: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Dashboard 1: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Reach: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Promotion Effect: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Conversion: 

	404008: Resource Not Fo

Error extracting CSV data for view ESG Insights Dashboard: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Gross by Genre: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Gross by Rating & Runtime: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Gross By Certification & Public Poles: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Gross By IMDB Ratings: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Total Invoice: 

	404008: Resource Not Found
		API version '2.7' does not support access to the given resource.
Error extracting CSV data for view Tax: 

	404008: Resource Not 